In [85]:
liste = [#"1erNovembre2.csv",
"1erNovembre3.csv",
"2018-11-02 16:41.csv",
"2NovembreBesac.csv"]

In [86]:
pds = []
for l in liste:
    pds.append(pd.read_csv(l, sep = ";"))

In [87]:
def cut_half(x):
    if x < .5:
        return 0
    else:
        return 1
pred = list((pds[0]["prediction"] + pds[1]["prediction"] + pds[2]["prediction"])/3)
prediction = [cut_half(x) for x in pred]

In [88]:
pds[0]["prediction"] = prediction
p =pds[0]

In [89]:
p.set_index("name", inplace=True)

In [92]:
p.to_csv("mix.csv",sep=";")

In [3]:
import warnings
warnings.filterwarnings("ignore")
import random
from copy import deepcopy as dp
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from tqdm import *
import os 
from os import system
import numpy as np 
from keras.preprocessing import image
from keras.initializers import RandomNormal as RN
from PIL import Image
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from matplotlib import pyplot as plt
from datetime import datetime
import glob
import numpy as np
import os.path as path
from scipy import misc

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime

In [14]:
size = 128
#img_width, img_height = size, size

data_train_path_target = "data_airbus_defi/train/"
data_test_path = "data_airbus_defi/test/"
input_shape = (size, size, 3)
#imput_shape = (size,size,1)

In [16]:
PATH = os.getcwd()
full = False
some = True
if full:
    training_size = float("inf")
    testing_size = float("inf")
elif some:
    training_size = 1000
    testing_size = float("inf")
else:
    training_size = 10
    testing_size = 10
    

train_path = PATH +"/" + data_train_path_target + "target/"
train_data_target = os.listdir(train_path)
x_train = []




#im_train = []
tdt = 0
for sample in (train_data_target):
    img_path = train_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_train.append(np.array(x))
    
    """
    im = Image.open(img_path)
    im_grey = im.convert('L') # convert the image to *greyscale*
    #im_bw = im_grey.point(lambda x: 0 if x<128 else 255, '1')
    im_array = np.array(im_grey)
    im_train.append(im_array)
    """
    tdt += 1
    
    if tdt > training_size:
        break

train_path = PATH +"/" + data_train_path_target + "other/"
train_data_other = os.listdir(train_path)
x_train2 = []
im_train2 = []
tdo = 0
for sample in (train_data_other):
    img_path = train_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_train2.append(np.array(x))
    """
    im = Image.open(img_path)
    im_grey = im.convert('L') # convert the image to *greyscale*
    #im_bw = im_grey.point(lambda x: 0 if x<128 else 255, '1')
    im_array = np.array(im_grey)
    im_train2.append(im_array)
    """
    tdo += 1
    if tdo > training_size:
        break

    
test_path = PATH+'/data_airbus_defi/test/'
test_data = os.listdir(test_path)
x_test = []
#im_test = []


td = 0
output = pd.DataFrame(columns=["name"])
test_data = [str(x) + ".jpg" for x in range(len(test_data))]
for sample in (test_data):
    output.append({"name": sample},ignore_index=True)
    #print(sample)
    img_path = test_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_test.append(np.array(x))
    """
    im = Image.open(img_path)
    im_grey = im.convert('L') # convert the image to *greyscale*
    #im_bw = im_grey.point(lambda x: 0 if x<128 else 255, '1')
    im_array = np.array(im_grey)
    im_test.append(im_array)
    """
    td+=1
    if td > testing_size:
        break
    
    
#test = np.array([np.array(x_t) for x_t in x_test])
# finally converting list into numpy array
x_train = np.array(x_train)
x_train2 = np.array(x_train2)
x_test = np.array(x_test) / 255.
XTRAIN = np.concatenate((x_train, x_train2), axis=0) / 255.

In [17]:
train_label1 = np.array([1] * x_train.shape[0] + [0] * x_train2.shape[0])

In [19]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
XTRAIN, train_labels = unison_shuffled_copies(XTRAIN,train_label1)
#IMTRAIN, im_labels = unison_shuffled_copies(IMTRAIN,train_labels)
#XTRAIN, train_labels  = zip(*random.shuffle(list(zip(XTRAIN,train_label1))))

In [18]:
system('say Data mélangée!')

0

In [20]:
EPOCHS = 150
BATCH_SIZE = 200

In [39]:
def cnn(size, n_layers):
    # INPUTS
    # size     - size of the input images
    # n_layers - number of layers
    # OUTPUTS
    # model    - compiled CNN

    # Define hyperparamters
    MIN_NEURONS = 3
    MAX_NEURONS = 6
    KERNEL = (3, 3)

    # Determine the # of neurons in each convolutional layer
    steps = np.floor(MAX_NEURONS / (n_layers + 1))
    nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
    nuerons = nuerons.astype(np.int32)

    # Define a model
    model = Sequential()

    # Add convolutional layers
    for i in range(0, n_layers):
        if i == 0:
            shape = (size[0], size[1], size[2])
            model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
        else:
            model.add(Conv2D(nuerons[i], KERNEL))

        model.add(Activation('relu'))

    # Add max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(MAX_NEURONS))
    model.add(Activation('relu'))

    # Add output layer
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # Print a summary of the model
    model.summary()

    return model

In [40]:
model = cnn(size=input_shape, n_layers=3)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 126, 126, 3)       84        
_________________________________________________________________
activation_11 (Activation)   (None, 126, 126, 3)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 124, 124, 4)       112       
_________________________________________________________________
activation_12 (Activation)   (None, 124, 124, 4)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 122, 122, 5)       185       
_________________________________________________________________
activation_13 (Activation)   (None, 122, 122, 5)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 61, 61, 5)         0         
__________

In [41]:
EPOCHS = 2
BATCH_SIZE = 200
PATIENCE = 10
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=0, mode='auto')

In [42]:
PATIENCE = 10
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=0, mode='auto')
LOG_DIRECTORY_ROOT = ''
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "TF" + now
tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)
callbacks = [early_stopping, tensorboard]

In [44]:
model.fit(XTRAIN, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks, verbose=0)

In [45]:
now

'20181103094232'

In [ ]:
## tensorboard --logdir TF2018......